# MSDS 7331 - Project 2
## Team: Dineen Parker, Dale Legband, Ryan Shuhart
collaboration site: https://github.com/rlshuhart/MSDS7331_Project_2

In [1]:
### Modules Used ###

# Data manipulation: pandas and numpy
import pandas as pd
import numpy as np

# Visualization: seaborn and matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# other
import time
import warnings

In [3]:
# load preprocessed data from previous lab

#data_url = "https://raw.githubusercontent.com/rlshuhart/MSDS7331_Project_1/master/cc_data/cc_data_processed.csv"
data_url = "../MSDS7331_Project_1/cc_data/cc_data_processed.csv"

cc_data = pd.read_csv(data_url, index_col='ID')
cc_data.info(0)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 1 to 30000
Data columns (total 26 columns):
EDUCATION                     30000 non-null int64
PAY_0_Sept                    30000 non-null int64
PAY_2_Aug                     30000 non-null int64
PAY_3_July                    30000 non-null int64
PAY_4_June                    30000 non-null int64
PAY_5_May                     30000 non-null int64
PAY_6_April                   30000 non-null int64
BILL_AMT1_Sept                30000 non-null int64
BILL_AMT2_Aug                 30000 non-null int64
BILL_AMT3_July                30000 non-null int64
BILL_AMT4_June                30000 non-null int64
BILL_AMT5_May                 30000 non-null int64
BILL_AMT6_April               30000 non-null int64
PAY_AMT1_Sept                 30000 non-null int64
PAY_AMT2_Aug                  30000 non-null int64
PAY_AMT3_July                 30000 non-null int64
PAY_AMT4_June                 30000 non-null int64
PAY_AMT5_May            

### Data Preparation (15 points total)
* [10 points] Define and prepare your class variables. Use proper variable
representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for
dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for
the analysis.

<font color='Red'>Not Complete</font>


* [5 points] Describe the final dataset that is used for classification/regression (include a
description of any newly formed variables you created).

<font color='Red'>Not Complete</font>


#### Convert data to numpy arrays for training

In [6]:
cc_data_target = cc_data['default payment next month']
cc_data_Xs = cc_data.drop('default payment next month', axis='columns')

# Column names for labeling
cc_data_Xs_names = cc_data.drop('default payment next month', axis='columns').columns

### Modeling and Evaluation (70 points total)
* [10 points] Choose and explain your evaluation metrics that you will use (i.e., accuracy,
precision, recall, F-measure, or any metric we have discussed). Why are the measure(s)
appropriate for analyzing the results of your modeling? Give a detailed explanation
backing up any assertions.

<font color='Red'>Not Complete</font>

    ROC Curves and AUC scores are good measures for imbalanced data scources. "The main goal for learning from imbalanced datasets is to improve the recall without hurting the precision...The F-value metric is one measure that combines the trade-offs of precision and recall, and outputs a single number reflecting the "goodness" of a classifier in the presence of rare classes. "
    https://www3.nd.edu/~dial/publications/chawla2005data.pdf

    From Wikipedia:
    "Precision (also called positive predictive value) is the fraction of retrieved instances that are relevant"
    "Recall (also known as sensitivity) is the fraction of relevant instances that are retrieved."
    https://en.wikipedia.org/wiki/Precision_and_recall
    



* [10 points] Choose the method you will use for dividing your data into training and
testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why
your chosen method is appropriate or use more than one method as appropriate. For
example, if you are using time series data then you should be using continuous training
and testing sets across time.

<font color='Red'>Not Complete</font>


* [20 points] Create three different classification/regression models for each task (e.g.,
random forest, KNN, and SVM for task one and the same or different algorithms for
task two). Two modeling techniques must be new (but the third could be SVM or
logistic regression). Adjust parameters as appropriate to increase generalization
performance using your chosen metric. You must investigate different parameters
of the algorithms!

<font color='Red'>Not Complete</font>


* [10 points] Analyze the results using your chosen method of evaluation. Use
visualizations of the results to bolster the analysis. Explain any visuals and analyze why
they are interesting to someone that might use this model.

<font color='Red'>Not Complete</font>


* [10 points] Discuss the advantages of each model for each classification task, if any. If
there are not advantages, explain why. Is any model better than another? Is the
difference significant with 95% confidence? Use proper statistical comparison methods.
You must use statistical comparison techniques—be sure they are appropriate for your
chosen method of validation as discussed in unit 7 of the course.

<font color='Red'>Not Complete</font>


* [10 points] Which attributes from your analysis are most important? Use proper
methods discussed in class to evaluate the importance of different attributes. Discuss
the results and hypothesize about why certain attributes are more important than others
for a given classification task.

<font color='Red'>Not Complete</font>

Deployment (5 points total)
* How useful is your model for interested parties (i.e., the companies or organizations
that might want to use it for prediction)? How would you measure the model's value if it
was used by these parties? How would your deploy your model for interested parties?
What other data should be collected? How often would the model need to be updated,
etc.?

<font color='Red'>Not Complete</font>
* The models provide a starting point for interested parties, identifying the most fruitful models with the data available. From this point, a conversation would be in order with the prospective model users about additional data that would enhance predictions, in theory, and requesting additional data to test and validate again. 
* To measure the value, one would implement the model and then advise customers that are going to default that they need to take some action to avoid that outcome. At the point where the CC company has fewer defaults and people have fewer defaults to struggle with, then you have a valid measurement: old default rage versus new default rate.
* Updates to the model would be determined by its overall impact to the customer and company, combined with new ideas that develop as the model is tweaked to come up with better predictions. At some point the tweaks will not significantly improve the prediction at which time the company may decide to just keep it as-is and move the resource to more productive problems that need to be tackled. 


Exceptional Work (10 points total)
* You have free reign to provide additional analyses.
* One idea: grid search parameters in a parallelized fashion and visualize the
performances across attributes. Which parameters are most significant for making a
good model for each classification algorithm?

<font color='Red'>Not Complete</font>

